In [1]:
from sagemaker import get_execution_role
import sagemaker
import boto3


role = get_execution_role()
aws_region = session.region_name
print(f'AWS region:{aws_region}')


session = boto3.session.Session()
sagemaker_session = sagemaker.session.Session(boto_session=session)
# #same function
# sess = sage.Session()

In [ ]:
model_path = "s3://{}/output".format(sagemaker.Session().default_bucket())
model_name = 'sagemaker-recsys-graph-inference'
# #same function：
# model_path = "s3://{}/output".format(sagemaker_session.default_bucket())
print(model_path)

In [ ]:
TrainJobName = 'sagemaker-recsys-graph-train'
training_params = {
    "RoleArn": role,
    "TrainingJobName": TrainJobName,
    "AlgorithmSpecification": {
        "TrainingImage": '{}.dkr.ecr.{}.amazonaws.com/sagemaker-pipe-demo:latest'.format(account, region),
        "TrainingInputMode": "Pipe"
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "{}".format(training_instance_type),
        "VolumeSizeInGB": 1
    },
    "InputDataConfig": [
        {
            "ChannelName": "training",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "{}".format(s3_training_input),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        }
    ],
    "OutputDataConfig": {
        "S3OutputPath": "{}".format(s3_model_output)
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 60 * 60
    }
}

In [ ]:
s3_model_url = model_path+'/sagemaker-recsys-graph-2020-11-17-03-15-32-694/model.tar.gz'
serving_image = '002224604296.dkr.ecr.us-east-1.amazonaws.com/sagemaker-recsys-graph-inference'
model_name = 'sagemaker-recsys-graph-inference'
serving_container_def = {
    'Image': serving_image,
    'ModelDataUrl': s3_model_url,
    'Mode': 'SingleModel',
    'Environment': { 'SM_MODEL_DIR' : '/opt/ml/model',
                     'RESNET_ARCH': 'resnet50' # 'resnet50' or 'resnet101'
                   }
}

create_model_response = sagemaker_session.create_model(name=model_name, 
                                                       role=role, 
                                                       container_defs=serving_container_def)

print(create_model_response)

In [ ]:
endpoint_config_name=f'{model_name}-endpoint-config'
print(endpoint_config_name)

In [ ]:
epc = sagemaker_session.create_endpoint_config(
    name=endpoint_config_name, 
    model_name=model_name, 
    initial_instance_count=1, 
    instance_type='ml.p2.xlarge')
print(epc)

In [ ]:
endpoint_name=f'{model_name}-endpoint'
print(endpoint_name)

In [ ]:
ep=sagemaker_session.create_endpoint(endpoint_name=endpoint_name, config_name=endpoint_config_name, wait=True)
print(ep)

In [ ]:
import base64
import json

data = {"instance":'中国银行'}
client = boto3.client('sagemaker-runtime')

body=json.dumps(data).encode('utf-8')

print(body)
    
response = client.invoke_endpoint(EndpointName=endpoint_name,
                                  ContentType="application/json",
                                  Accept="application/json",
                                  Body=body)

# print(response[''])
# body=response['Body'].read()
# msg=body.decode('utf-8')

In [ ]:
print(response['Body'].read())

In [2]:
# Lets set up our SageMaker notebook instance for local mode.
!/bin/bash ./utils/setup.sh

The user has root access.
nvidia-docker2 already installed. We are good to go!
SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


In [22]:
from sagemaker.estimator import Estimator

hyperparameters = {'train-steps': 100}

instance_type = 'local_gpu'

estimator = Estimator(role=role,
                      instance_count=1,
                      instance_type=instance_type,
                      image_name='sagemaker-recsys-graph:latest',
                      image_uri='002224604296.dkr.ecr.us-east-1.amazonaws.com/sagemaker-recsys-graph',
                      hyperparameters=hyperparameters)

estimator.fit()

Creating tmppl6l83bh_algo-1-3fqql_1 ... 
Attaching to tmppl6l83bh_algo-1-3fqql_12mdone
algo-1-3fqql_1  | sys path is ['/opt/ml/code', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/src/dglke/python', '/usr/lib/python3/dist-packages']
algo-1-3fqql_1  | Using backend: pytorch
algo-1-3fqql_1  | Reading train triples....
algo-1-3fqql_1  | Finished. Read 3603022 train triples.
algo-1-3fqql_1  | |Train|: 3603022
algo-1-3fqql_1  | /usr/local/lib/python3.6/dist-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
algo-1-3fqql_1  |   warnings.warn(msg, warn_type)
algo-1-3fqql_1  | Total initialize time 8.227 seconds
algo-1-3fqql_1  | [proc 0][Train](2/200) average pos_loss: 2.8938835859298706
algo-1-3fqql_1  | [proc 0][Train](2/200) average neg_loss: 0.4505869150161743
algo-1-3fqql_1  | [proc 0][Train](2/200) average loss: 1.6722352504730225
algo-1-

algo-1-3fqql_1  | [proc 0][Train](34/200) average pos_loss: 3.2416456937789917
algo-1-3fqql_1  | [proc 0][Train](34/200) average neg_loss: 0.6891139298677444
algo-1-3fqql_1  | [proc 0][Train](34/200) average loss: 1.9653798341751099
algo-1-3fqql_1  | [proc 0][Train](34/200) average regularization: 3.5445538742351346e-05
algo-1-3fqql_1  | [proc 0][Train] 2 steps take 0.044 seconds
algo-1-3fqql_1  | [proc 0]sample: 0.018, forward: 0.011, backward: 0.010, update: 0.005
algo-1-3fqql_1  | [proc 0][Train](36/200) average pos_loss: 3.0987812280654907
algo-1-3fqql_1  | [proc 0][Train](36/200) average neg_loss: 0.834689199924469
algo-1-3fqql_1  | [proc 0][Train](36/200) average loss: 1.9667352437973022
algo-1-3fqql_1  | [proc 0][Train](36/200) average regularization: 3.194000782968942e-05
algo-1-3fqql_1  | [proc 0][Train] 2 steps take 0.028 seconds
algo-1-3fqql_1  | [proc 0]sample: 0.002, forward: 0.011, backward: 0.010, update: 0.005
algo-1-3fqql_1  | [proc 0][Train](38/200) average pos_loss: 

algo-1-3fqql_1  | [proc 0][Train](66/200) average pos_loss: 3.897414803504944
algo-1-3fqql_1  | [proc 0][Train](66/200) average neg_loss: 0.8727235198020935
algo-1-3fqql_1  | [proc 0][Train](66/200) average loss: 2.385069251060486
algo-1-3fqql_1  | [proc 0][Train](66/200) average regularization: 3.7043570046080276e-05
algo-1-3fqql_1  | [proc 0][Train] 2 steps take 0.045 seconds
algo-1-3fqql_1  | [proc 0]sample: 0.018, forward: 0.011, backward: 0.010, update: 0.005
algo-1-3fqql_1  | [proc 0][Train](68/200) average pos_loss: 3.8845053911209106
algo-1-3fqql_1  | [proc 0][Train](68/200) average neg_loss: 0.8023785650730133
algo-1-3fqql_1  | [proc 0][Train](68/200) average loss: 2.343441963195801
algo-1-3fqql_1  | [proc 0][Train](68/200) average regularization: 3.693341204780154e-05
algo-1-3fqql_1  | [proc 0][Train] 2 steps take 0.028 seconds
algo-1-3fqql_1  | [proc 0]sample: 0.002, forward: 0.011, backward: 0.010, update: 0.005
algo-1-3fqql_1  | [proc 0][Train](70/200) average pos_loss: 3.

algo-1-3fqql_1  | [proc 0][Train](98/200) average pos_loss: 4.369554281234741
algo-1-3fqql_1  | [proc 0][Train](98/200) average neg_loss: 1.0272396802902222
algo-1-3fqql_1  | [proc 0][Train](98/200) average loss: 2.698396921157837
algo-1-3fqql_1  | [proc 0][Train](98/200) average regularization: 4.1697277993080206e-05
algo-1-3fqql_1  | [proc 0][Train] 2 steps take 0.043 seconds
algo-1-3fqql_1  | [proc 0]sample: 0.017, forward: 0.011, backward: 0.010, update: 0.005
algo-1-3fqql_1  | [proc 0][Train](100/200) average pos_loss: 4.602865695953369
algo-1-3fqql_1  | [proc 0][Train](100/200) average neg_loss: 0.813959002494812
algo-1-3fqql_1  | [proc 0][Train](100/200) average loss: 2.708412289619446
algo-1-3fqql_1  | [proc 0][Train](100/200) average regularization: 3.788250978686847e-05
algo-1-3fqql_1  | [proc 0][Train] 2 steps take 0.029 seconds
algo-1-3fqql_1  | [proc 0]sample: 0.002, forward: 0.012, backward: 0.010, update: 0.005
algo-1-3fqql_1  | [proc 0][Train](102/200) average pos_loss:

algo-1-3fqql_1  | [proc 0][Train](130/200) average pos_loss: 4.84494948387146
algo-1-3fqql_1  | [proc 0][Train](130/200) average neg_loss: 1.045610249042511
algo-1-3fqql_1  | [proc 0][Train](130/200) average loss: 2.945279836654663
algo-1-3fqql_1  | [proc 0][Train](130/200) average regularization: 4.3379141061450355e-05
algo-1-3fqql_1  | [proc 0][Train] 2 steps take 0.042 seconds
algo-1-3fqql_1  | [proc 0]sample: 0.016, forward: 0.011, backward: 0.010, update: 0.005
algo-1-3fqql_1  | [proc 0][Train](132/200) average pos_loss: 5.23169207572937
algo-1-3fqql_1  | [proc 0][Train](132/200) average neg_loss: 1.1098530292510986
algo-1-3fqql_1  | [proc 0][Train](132/200) average loss: 3.1707725524902344
algo-1-3fqql_1  | [proc 0][Train](132/200) average regularization: 4.0570952478447e-05
algo-1-3fqql_1  | [proc 0][Train] 2 steps take 0.028 seconds
algo-1-3fqql_1  | [proc 0]sample: 0.002, forward: 0.011, backward: 0.010, update: 0.005
algo-1-3fqql_1  | [proc 0][Train](134/200) average pos_loss

algo-1-3fqql_1  | [proc 0][Train](162/200) average pos_loss: 5.413949728012085
algo-1-3fqql_1  | [proc 0][Train](162/200) average neg_loss: 1.0832771062850952
algo-1-3fqql_1  | [proc 0][Train](162/200) average loss: 3.2486133575439453
algo-1-3fqql_1  | [proc 0][Train](162/200) average regularization: 4.201701631245669e-05
algo-1-3fqql_1  | [proc 0][Train] 2 steps take 0.042 seconds
algo-1-3fqql_1  | [proc 0]sample: 0.016, forward: 0.011, backward: 0.010, update: 0.005
algo-1-3fqql_1  | [proc 0][Train](164/200) average pos_loss: 5.338684797286987
algo-1-3fqql_1  | [proc 0][Train](164/200) average neg_loss: 1.1608354449272156
algo-1-3fqql_1  | [proc 0][Train](164/200) average loss: 3.2497600317001343
algo-1-3fqql_1  | [proc 0][Train](164/200) average regularization: 3.854392889479641e-05
algo-1-3fqql_1  | [proc 0][Train] 2 steps take 0.028 seconds
algo-1-3fqql_1  | [proc 0]sample: 0.002, forward: 0.011, backward: 0.010, update: 0.005
algo-1-3fqql_1  | [proc 0][Train](166/200) average pos

algo-1-3fqql_1  | [proc 0][Train](194/200) average pos_loss: 5.611870288848877
algo-1-3fqql_1  | [proc 0][Train](194/200) average neg_loss: 1.3282113075256348
algo-1-3fqql_1  | [proc 0][Train](194/200) average loss: 3.470040798187256
algo-1-3fqql_1  | [proc 0][Train](194/200) average regularization: 4.531420927378349e-05
algo-1-3fqql_1  | [proc 0][Train] 2 steps take 0.047 seconds
algo-1-3fqql_1  | [proc 0]sample: 0.020, forward: 0.012, backward: 0.010, update: 0.005
algo-1-3fqql_1  | [proc 0][Train](196/200) average pos_loss: 6.032917261123657
algo-1-3fqql_1  | [proc 0][Train](196/200) average neg_loss: 1.0730957984924316
algo-1-3fqql_1  | [proc 0][Train](196/200) average loss: 3.5530065298080444
algo-1-3fqql_1  | [proc 0][Train](196/200) average regularization: 4.6991059207357466e-05
algo-1-3fqql_1  | [proc 0][Train] 2 steps take 0.028 seconds
algo-1-3fqql_1  | [proc 0]sample: 0.002, forward: 0.012, backward: 0.010, update: 0.005
algo-1-3fqql_1  | [proc 0][Train](198/200) average pos

In [23]:
result = estimator.deploy(1, instance_type)

Attaching to tmp6azdd0q4_algo-1-xuics_1
algo-1-xuics_1  | Using backend: pytorch
algo-1-xuics_1  | sys path is ['/opt/ml/code', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/src/dglke/python', '/usr/lib/python3/dist-packages']
http://212.129.155.247/fasthan/fasthan_base.zip not found in cache, downloading to /tmp/tmpt0fe3eq_
100% 144M/144M [00:20<00:00, 6.89MB/s] 
algo-1-xuics_1  | Finish download from http://212.129.155.247/fasthan/fasthan_base.zip
algo-1-xuics_1  | Copy file to /root/.fastNLP/fasthan/fasthan_base
algo-1-xuics_1  | loading vocabulary file /root/.fastNLP/fasthan/fasthan_base/vocab.txt
algo-1-xuics_1  | Load pre-trained BERT parameters from file /root/.fastNLP/fasthan/fasthan_base/model.bin.
tmp6azdd0q4_algo-1-xuics_1 exited with code 0
Aborting on container exit...


RuntimeError: Giving up, endpoint didn't launch correctly

In [14]:
# 单独测试代码
!pip install -r requirements

pip 20.0.2 from /home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/pip (python 3.6)


In [ ]:
import kg
import encoding
kg = kg.Kg('kg')
encoder = encoding.encoding(kg)